In [ ]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10,5)


In [ ]:
# load dataset
df = pd.read_csv("inventory.csv")
df.head()

In [ ]:
#check columns
df.columns

In [ ]:
# mapping columns 

col_item = "Item"
col_demand = "Annual_Demand"
col_unit_price = "Unit_Price"
col_order_cost = "Ordering_Cost"
col_hold_cost = "Holding_Cost"
col_lead_time = "Lead_Time"
col_safety_stock = "Safety_Stock"
col_avg_inventory = "Average_Inventory"
# ---------------------------------------------------------
print("Column mapping is set!")


In [ ]:
# basic cleaning
df = df.drop_duplicates()

# Fill missing numeric values
for col in df.select_dtypes(include=['int64','float64']).columns:
    df[col] = df[col].fillna(df[col].median())

# Fill missing categorical values
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].fillna(df[col].mode()[0])

df.head()

In [ ]:
# ABC analysis

df["annual_value"] = df[col_demand] * df[col_unit_price]
df_sorted = df.sort_values("annual_value", ascending=False)
df_sorted["cum_percent"] = df_sorted["annual_value"].cumsum() / df_sorted["annual_value"].sum()

def classify_abc(x):
    if x <= 0.7:
        return "A"
    elif x <= 0.9:
        return "B"
    else:
        return "C"

df_sorted["ABC"] = df_sorted["cum_percent"].apply(classify_abc)
df_sorted[["Item","annual_value","ABC"]].head(10)

In [ ]:
# EOQ Calculation

df["EOQ"] = np.sqrt((2 * df[col_demand] * df[col_order_cost]) / df[col_hold_cost])
df[["Item","EOQ"]].head(10)

In [ ]:
# Reorder Point

df["daily_demand"] = df[col_demand] / 365
df["ROP"] = (df["daily_demand"] * df[col_lead_time]) + df[col_safety_stock]
df[["Item","ROP"]].head(10)

In [ ]:
# inventory turnover

df["usage_value"] = df[col_demand] * df[col_unit_price]
df["inventory_turnover"] = df["usage_value"] / df[col_avg_inventory]
df[["Item","inventory_turnover"]].head(10)

In [ ]:
# carrying cost

df["carrying_cost"] = df[col_avg_inventory] * df[col_hold_cost]
df[["Item","carrying_cost"]].head(10)

In [ ]:
# Summary table

summary = df[[col_item, "ABC", "EOQ", "ROP", "inventory_turnover", "carrying_cost"]]
summary.head(20)